# Experiment: Which drugs are "similar", from prescription data?

In the USA the social program for prescription drugs is the "Medicare Part D Prescription Drug" program. Data regarding the prescribed drugs per provider and drug are available at https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider-and-drug/

On this website you can select the columns you want. I downloaded only the code for the drug provider (`Prscrbr_NPI`), the type of provider (`Prscrbr_Type`), the generic name for the drug (`Gnrc_Name`) and the total number of claims made by this provider (`Tot_Clms`):

<center><img src='./image.png' width=50%></img></center>

As a result, I got a 197MB zip file. I made a folder named `data` and placed the file there. In the previous notebook we did some Exploratory Data Analysis (EDA) and filtering, and saved the dataset in a more efficient format (Parquet). Lets read it.

In [11]:
from pathlib import Path

import numpy as np
import pandas as pd

from utils import DATA_DIR, load_clean_data, load_data

In [12]:
# df = load_clean_data()
df = load_data()

c:\Users\eduya\github-classroom\4-semestre\materias-aula\machine-learning\machine_learning\labs\01_SVD\labs\data\Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.csv not found, extracting zip file
Loading data from c:\Users\eduya\github-classroom\4-semestre\materias-aula\machine-learning\machine_learning\labs\01_SVD\labs\data\Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.csv


Let's create a sparse matrix from the dataframe. The trick is to convert the columns of interest to categorical data, so that Pandas will identify the unique doctors and drugs, and make an integer code for each one, from zero to the number of unique items minus one.

In [13]:
df = df.reset_index(drop=True)

In [14]:
df['Prscrbr_NPI'] = df['Prscrbr_NPI'].astype('category')
df['Gnrc_Name'] = df['Gnrc_Name'].astype('category')
df['Prscrbr_Type'] = df['Prscrbr_Type'].astype('category')
df['Tot_Clms'] = df['Tot_Clms'].astype(float)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25869521 entries, 0 to 25869520
Data columns (total 4 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Prscrbr_NPI   category
 1   Prscrbr_Type  category
 2   Gnrc_Name     category
 3   Tot_Clms      float64 
dtypes: category(3), float64(1)
memory usage: 435.1 MB


Let's see the categorical encoding in action. What are the drugs?

In [16]:
df['Gnrc_Name'].cat.categories

Index(['0.9 % Sodium Chloride', 'Aa 4.25%/Calcium/Lytes/Dex 10%',
       'Aa 5 %/Calcium/Lytes/Dext 20 %', 'Abacavir Sulfate',
       'Abacavir Sulfate/Lamivudine', 'Abacavir/Dolutegravir/Lamivudi',
       'Abacavir/Lamivudine/Zidovudine', 'Abaloparatide', 'Abatacept',
       'Abatacept/Maltose',
       ...
       'Zileuton', 'Zinc/Copper/Manganese/Selenium', 'Ziprasidone Hcl',
       'Ziprasidone Mesylate', 'Zoledronic Ac/Mannitol/0.9nacl',
       'Zoledronic Acid', 'Zoledronic Acid/Mannitol-Water', 'Zolmitriptan',
       'Zolpidem Tartrate', 'Zonisamide'],
      dtype='object', length=1757)

So each drug is associated with an index number. In this case, "0.9 % Sodium Chloride" is drug number zero, "Abacavir Sulfate" is drug number one, and so on.

Let's see what is stored now in the dataframe:

In [17]:
df['Gnrc_Name'].head()

0                  Cefazolin Sodium
1                Ceftriaxone Sodium
2                       Amoxicillin
3    Bictegrav/Emtricit/Tenofov Ala
4           Doxycycline Monohydrate
Name: Gnrc_Name, dtype: category
Categories (1757, object): ['0.9 % Sodium Chloride', 'Aa 4.25%/Calcium/Lytes/Dex 10%', 'Aa 5 %/Calcium/Lytes/Dext 20 %', 'Abacavir Sulfate', ..., 'Zoledronic Acid/Mannitol-Water', 'Zolmitriptan', 'Zolpidem Tartrate', 'Zonisamide']

Looks like the same as before. But what about the equivalent codes?

In [18]:
df['Gnrc_Name'].cat.codes.head()

0    274
1    291
2     87
3    188
4    504
dtype: int16

Do the codes and names really correspond to each other? Sure they do! Look at this:

In [19]:
for code in df['Gnrc_Name'].cat.codes.head():
    print(f'code is {code}, name is {df['Gnrc_Name'].cat.categories[code]}')

SyntaxError: f-string: unmatched '[' (2434141049.py, line 2)

Now we can make a matrix of the data. We will use the doctor NPI as the row index and the drug name as the column index. The value of each cell will be the number of prescriptions.

Mathematically, this is a matrix like any other. Computationally, we will make a sparse matrix in the CSR - Compressed Sparse Row - format (https://en.wikipedia.org/wiki/Sparse_matrix)

In [20]:
row_index = df['Prscrbr_NPI'].cat.codes
num_rows = df['Prscrbr_NPI'].cat.categories.size

col_index = df['Gnrc_Name'].cat.codes
num_cols = df['Gnrc_Name'].cat.categories.size

values = df['Tot_Clms'].values

In [21]:
from scipy.sparse import csr_matrix

sparse_matrix = csr_matrix(
    (values, (row_index, col_index)),
    shape=(num_rows, num_cols),
)

In [22]:
sparse_matrix.shape

(1057566, 1757)

Now we are ready for the Singular Value Decomposition (SVD). What we want to do is to decompose our matrix $\bm{X} \in \mathbb{R}^{m \times n}$ of doctors per medication (where $m$ is the number of doctors and $n$ is the number of medications, $m \gg n$) into a product of three matrices: $\bm{U} \in \mathbb{R}^{m \times m}$, $S \in \mathbb{R}^{m \times n}$, and $V^{T} \in \mathbb{R}^{n \times n}$ such that:

- $\bm{X} = \bm{U} \bm{S} \bm{V}^{T}$
- Matrices $\bm{U}$ and $\bm{V}$ are unitary: 
    - $\bm{U}^{T} \bm{U} = \bm{U} \bm{U}^{T} = \bm{I}^{(m)}$ is the identity matrix of size $m$
    - $\bm{V}^{T} \bm{V} = \bm{V} \bm{V}^{T} = \bm{I}^{(n)}$ is the identity matrix of size $n$
- Matrix $S$ is formed by two vertically stacked blocks (since $m \gg n$):
    - The top block is a diagonal matrix $\Sigma \in \mathbb{R}^{n \times n}$. The diagonal elements are all non-negative (i.e. are positive or zero) and placed in decreasing order. These numbers are the *singular values* of $\bm{X}$
    - The bottom block is a matrix of zeros, of size $(m - n) \times n$

In [23]:
from scipy.sparse.linalg import svds

But we don't want *all* columns of $\bm{U}$ and rows of $\bm{V}$. We only want a few of them to investigate! So lets set a number of components to obtain, and apply the *truncated* SVD:

In [28]:
num_components = 5

In [29]:
U, sigma, Vt = svds(sparse_matrix, k=num_components)

In [30]:
U.shape, sigma.shape, Vt.shape

((1057566, 5), (5,), (5, 1757))

So now we have `num_components` columns in $\bm{U}$ and rows in $V^{T}$. Since this is an SVD, we already know that the columns of $\bm{U}$ are unit vectors, and all orthogonal to each other. By computing $\bm{U}^{T} \bm{U}$ we calculate all dot products between the columns of $\bm{U}$:

In [32]:

(U.T @ U).round(2)

array([[ 1., -0.,  0., -0.,  0.],
       [-0.,  1.,  0., -0.,  0.],
       [ 0.,  0.,  1.,  0., -0.],
       [-0., -0.,  0.,  1., -0.],
       [ 0.,  0., -0., -0.,  1.]])

As we can see, the matrix product shows that the dot product between $\bm{u}^{(i)}$ and $\bm{v}^{(j)}$ is zero if $i \neq j$ and 1 if $i = j$: $\bm{U}^{T} \bm{U} = \bm{I}_{k \times k}$. The same applies to the rows of $\bm{V}^{T}$, that is, $V^{T} V = V^{T} (V^{T})^{T} = I_{k \times k}$

In [20]:
(Vt @ Vt.T).round(2)

array([[ 1.,  0., -0., -0., -0.],
       [ 0.,  1., -0.,  0.,  0.],
       [-0., -0.,  1., -0., -0.],
       [-0.,  0., -0.,  1., -0.],
       [-0.,  0., -0., -0.,  1.]])

What is interesting is the *rows* of $\bm{U}$ and the *columns* of $\bm{V}^{T}$:

- Each row of $\bm{U}$ is a *reduced representation* of a doctor.
- Each column of $\bm{V}^{T}$ is a *reduced representation* of a drug.

We can use these reduced representations in many applications:

- Finding *outliers*: doctors whose prescription pattern is way off in regards to their counterparts. Say a doctor is prescribing an enormous amount of opioids but not a lot of anything else. As we find the reduced representation for that doctor, we can compare it to all other doctors (via *cosine similarity*) and see if this doctor stands out from the cohort. If so, it could be an indication of fraudulent activity.
- Predicting future behaviour: given the current pattern of prescriptions of a doctor, we may:
    - Find its reduced representation
    - Reconstruct the estimated prescriptions from the reduced representation: the difference to the actual presciption data for the doctor may indicate drugs that this physician is expected to prescribe at some point.
- Understanding the set of doctors and the set of drugs: We can look for similar groups of doctors and drugs, in an activity called *clustering*.

Lets look at the doctor and drugs data to explore patterns. Save the drugs features (the reduced representation in $\bm{V}$) in a tab-separated file, and the drug names in another text file:

In [33]:
np.savetxt(DATA_DIR / 'drug_features.tsv', Vt.T, delimiter='\t')

In [34]:
np.savetxt(
    DATA_DIR / 'drug_labels.tsv',
    df['Gnrc_Name'].cat.categories.values,
    delimiter='\t',
    fmt='%s',
    encoding='utf-8',
)

Lets also save the doctors' reduced representations $\bm{U}$ and their specialties:

In [35]:
np.savetxt(DATA_DIR / 'physician_features.tsv', U, delimiter='\t')

In [36]:
df['Prscrbr_Type'].cat.categories.values

array(['Acupuncturist', 'Addiction Medicine',
       'Adult Congenital Heart Disease',
       'Advanced Heart Failure and Transplant Cardiology',
       'Advanced Practice Dental Therapist', 'Allergy/ Immunology',
       'Ambulatory Surgical Center', 'Anesthesiology',
       'Assisted Living Facility', 'Behavior Analyst', 'Cardiac Surgery',
       'Cardiology', 'Case Manager/Care Coordinator',
       'Certified Clinical Nurse Specialist', 'Certified Nurse Midwife',
       'Certified Registered Nurse Anesthetist (CRNA)', 'Chiropractic',
       'Chore Provider', 'Clinic or Group Practice', 'Clinic/Center',
       'Clinical Cardiac Electrophysiology',
       'Clinical Medical Laboratory', 'Clinical Neuropsychologist',
       'Clinical Pharmacology', 'Colon & Rectal Surgery',
       'Colorectal Surgery (Proctology)', 'Community Health Worker',
       'Community/Behavioral Health', 'Contractor', 'Counselor',
       'Critical Care (Intensivists)', 'Dental Assistant',
       'Dental Hygienist

In [37]:
df_npi_type = df \
    .loc[:, ['Prscrbr_NPI', 'Prscrbr_Type']] \
    .groupby(['Prscrbr_NPI', 'Prscrbr_Type'], observed=True) \
    .count() \
    .reset_index() \
    .set_index('Prscrbr_NPI')

df_npi = \
    pd.DataFrame(
        df['Prscrbr_NPI'].cat.categories,
        columns=['Prscrbr_NPI'],
    ) \
    .reset_index() \
    .set_index('Prscrbr_NPI')

df_npi = df_npi \
    .join(df_npi_type) \
    .reset_index() \
    .set_index('index')

In [38]:
df_npi

,Prscrbr_NPI,Prscrbr_Type
index,,
0,1003000126,Internal Medicine
1,1003000142,Anesthesiology
2,1003000167,Dentist
3,1003000423,Obstetrics & Gynecology
4,1003000480,General Surgery
...,...,...
1057561,1992999437,Emergency Medicine
1057562,1992999551,Internal Medicine
1057563,1992999650,Dentist


In [39]:
df_npi.to_csv(DATA_DIR / 'physician_labels.tsv', sep='\t', index=False)

In [40]:
np.savetxt(
    DATA_DIR / 'physician_labels.tsv',
    df_npi,
    delimiter='\t',
    fmt='%s',
    encoding='utf-8',
)